# Populate download_uuid in release_external_ids using phash matching

This notebook populates the `download_uuid` column in `release_external_ids` by:
1. Getting all existing external ID mappings (release -> stashapp scene)
2. For each release, finding the video download with a phash
3. Finding the corresponding Stashapp scene by phash
4. Updating the external ID row with the matching download_uuid

In [ ]:
import os
import polars as pl
from dotenv import load_dotenv
import sys
import psycopg2

load_dotenv()

sys.path.append(os.path.dirname(os.getcwd()))

from libraries.client_stashapp import StashAppClient, get_stashapp_client
from libraries.client_culture_extractor import ClientCultureExtractor

In [ ]:
# Initialize clients
stash_client = StashAppClient()
stash_raw = get_stashapp_client()

# Build connection string for CE client
host = os.environ.get("CE_DB_HOST")
port = os.environ.get("CE_DB_PORT")
db = os.environ.get("CE_DB_NAME")
user = os.environ.get("CE_DB_USERNAME")
pw = os.environ.get("CE_DB_PASSWORD")
connection_string = f"dbname={db} user={user} password={pw} host={host} port={port}"

ce_client = ClientCultureExtractor(connection_string)

In [ ]:
# Get all release_external_ids that need download_uuid populated
# These are rows where download_uuid is NULL and target_system is 'stashapp'

conn = psycopg2.connect(
    host=os.environ.get("CE_DB_HOST"),
    port=os.environ.get("CE_DB_PORT"),
    database=os.environ.get("CE_DB_NAME"),
    user=os.environ.get("CE_DB_USERNAME"),
    password=os.environ.get("CE_DB_PASSWORD"),
)

cursor = conn.cursor()
cursor.execute("""
    SELECT 
        rei.uuid as external_id_uuid,
        rei.release_uuid,
        r.name as release_name,
        rei.external_id as stashapp_id,
        ts.name as target_system,
        s.name as site_name
    FROM release_external_ids rei
    JOIN target_systems ts ON rei.target_system_uuid = ts.uuid
    JOIN releases r ON rei.release_uuid = r.uuid
    JOIN sites s ON r.site_uuid = s.uuid
    WHERE rei.download_uuid IS NULL
    AND ts.name = 'stashapp'
""")

external_ids_to_update = cursor.fetchall()
print(f"Found {len(external_ids_to_update)} external IDs that need download_uuid populated")

# Convert to DataFrame for easier manipulation
external_ids_df = pl.DataFrame(
    external_ids_to_update,
    schema=["external_id_uuid", "release_uuid", "release_name", "stashapp_id", "target_system", "site_name"],
    orient="row"
)
external_ids_df

In [ ]:
# Get all video downloads with phash for the releases that have external IDs
release_uuids = external_ids_df["release_uuid"].to_list()

cursor.execute("""
    SELECT 
        d.uuid as download_uuid,
        d.release_uuid,
        d.file_type,
        d.content_type,
        d.file_metadata->>'phash' as phash,
        d.file_metadata->>'oshash' as oshash
    FROM downloads d
    WHERE d.release_uuid = ANY(%s::uuid[])
    AND d.file_type = 'video'
    AND d.content_type = 'scene'
    AND d.file_metadata->>'phash' IS NOT NULL
""", (release_uuids,))

downloads = cursor.fetchall()
print(f"Found {len(downloads)} video downloads with phash")

downloads_df = pl.DataFrame(
    downloads,
    schema=["download_uuid", "release_uuid", "file_type", "content_type", "phash", "oshash"],
    orient="row"
)
downloads_df

In [ ]:
# Get all Stashapp scenes that are referenced in the external IDs
stashapp_ids = external_ids_df["stashapp_id"].to_list()

# Get scenes from Stashapp with their fingerprints, stash_ids, and studio
scenes_fragment = """
    id
    title
    files {
        id
        fingerprints {
            type
            value
        }
    }
    stash_ids {
        endpoint
        stash_id
    }
    studio {
        id
        name
        parent_studio {
            id
            name
        }
    }
"""

STASHDB_ENDPOINT = "https://stashdb.org/graphql"

# Fetch scenes in batches to avoid overwhelming the API
stashapp_scenes = []
for stashapp_id in stashapp_ids:
    try:
        scene = stash_raw.find_scene(int(stashapp_id), fragment=scenes_fragment)
        if scene:
            # Extract phash from fingerprints
            phash = None
            oshash = None
            for file in scene.get("files", []):
                for fp in file.get("fingerprints", []):
                    if fp["type"] == "phash":
                        phash = fp["value"]
                    elif fp["type"] == "oshash":
                        oshash = fp["value"]
            
            # Extract StashDB ID from stash_ids
            stashdb_id = None
            for stash_id in scene.get("stash_ids", []):
                if stash_id.get("endpoint") == STASHDB_ENDPOINT:
                    stashdb_id = stash_id.get("stash_id")
                    break
            
            # Extract studio name (prefer parent studio if available)
            studio = scene.get("studio")
            studio_name = None
            if studio:
                parent = studio.get("parent_studio")
                if parent:
                    studio_name = f"{parent.get('name')} / {studio.get('name')}"
                else:
                    studio_name = studio.get("name")
            
            stashapp_scenes.append({
                "stashapp_id": str(scene["id"]),
                "stashapp_title": scene.get("title"),
                "stashapp_studio": studio_name,
                "stashapp_phash": phash,
                "stashapp_oshash": oshash,
                "stashdb_id": stashdb_id,
            })
    except Exception as e:
        print(f"Error fetching scene {stashapp_id}: {e}")

stashapp_scenes_df = pl.DataFrame(stashapp_scenes)
print(f"Fetched {len(stashapp_scenes_df)} Stashapp scenes")
print(f"Scenes with StashDB ID: {stashapp_scenes_df.filter(pl.col('stashdb_id').is_not_null()).height}")
stashapp_scenes_df

In [ ]:
# Join external IDs with downloads and Stashapp scenes
# Match by: external_id -> stashapp_id -> stashapp_oshash -> download_oshash -> download_uuid

# First join external IDs with stashapp scenes
matched_df = external_ids_df.join(
    stashapp_scenes_df,
    on="stashapp_id",
    how="left"
)

# Then join with downloads by oshash
matched_df = matched_df.join(
    downloads_df.select(["download_uuid", "release_uuid", "oshash"]).rename({"oshash": "download_oshash"}),
    on="release_uuid",
    how="left"
)

# Filter to rows where oshash matches
matched_df = matched_df.filter(
    pl.col("stashapp_oshash") == pl.col("download_oshash")
)

print(f"Found {len(matched_df)} matches by oshash")
matched_df.select([
    "release_name", "stashapp_title", "stashapp_studio", "site_name",
    "stashapp_oshash", "download_oshash", "stashdb_id", "download_uuid"
])

In [ ]:
# Review the matches before applying
print("Matches to be applied:")
print(f"Total external IDs to update: {len(matched_df)}")
print(f"\nBy site:")
print(matched_df.group_by("site_name").agg(pl.count()).sort("count", descending=True))

# Check for any external IDs that couldn't be matched
unmatched_df = external_ids_df.join(
    matched_df.select(["external_id_uuid"]),
    on="external_id_uuid",
    how="anti"
)
print(f"\nExternal IDs that couldn't be matched: {len(unmatched_df)}")
if len(unmatched_df) > 0:
    print(unmatched_df)

In [ ]:
# Apply the updates
update_count = 0
error_count = 0

for row in matched_df.iter_rows(named=True):
    external_id_uuid = row["external_id_uuid"]
    download_uuid = row["download_uuid"]
    
    if download_uuid is None:
        print(f"Skipping {external_id_uuid} - no download_uuid")
        continue
    
    try:
        cursor.execute("""
            UPDATE release_external_ids
            SET download_uuid = %s, last_updated = NOW()
            WHERE uuid = %s
        """, (download_uuid, external_id_uuid))
        update_count += 1
        print(f"✓ Updated {external_id_uuid} with download_uuid {download_uuid}")
    except Exception as e:
        error_count += 1
        print(f"✗ Error updating {external_id_uuid}: {e}")

conn.commit()
print(f"\nUpdated {update_count} rows, {error_count} errors")

In [ ]:
# Verify the updates
cursor.execute("""
    SELECT 
        COUNT(*) as total,
        COUNT(download_uuid) as with_download_uuid,
        COUNT(*) - COUNT(download_uuid) as without_download_uuid
    FROM release_external_ids rei
    JOIN target_systems ts ON rei.target_system_uuid = ts.uuid
    WHERE ts.name = 'stashapp'
""")
result = cursor.fetchone()
print(f"Total stashapp external IDs: {result[0]}")
print(f"With download_uuid: {result[1]}")
print(f"Without download_uuid: {result[2]}")

In [ ]:
# Close connections
cursor.close()
conn.close()